In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Preprocessing
!ls

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1vvRy60J06fZq-dj8EPVTkgHU4gq8OCqU/Capstone-KPMG/Preprocessing
Attractions.ipynb		 I90.ipynb
combine_data.ipynb		 map_request_api.py
Combine.ipynb			 OBSOLETE_Geo.ipynb
Data				 OBSOLETE_remoteDataList
EV_station_count_distance.ipynb  __pycache__
gas_station_w_traffic.csv	 traffic_population_crime_combination.ipynb
HighwayExitDistance.ipynb


In [2]:
import pandas as pd
from map_request_api import *

In [7]:
# import map_request_api as mrp
# print(mrp.__version__)
import json
print(json.__version__)

2.0.9


# Generate gas station to exit data for stations next to I5 Exit 1A to Exit 108

In [ ]:
# Load exit data
exits = pd.read_csv('Data/I5Exits.csv')

# End 108, Start 1A
end_lat = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.y']
end_long = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.x']
start_lat = exits[exits['attributes.ExitNumber']== '1 A'].iloc[0]['geometry.y']
start_long = exits[exits['attributes.ExitNumber']== '1 A'].iloc[0]['geometry.x']

In [ ]:
# get route/corridor
url = build_search_url_latlong(str(start_lat), str(start_long), str(end_lat), str(end_long))
result = get_result(url)
sessionId = result['route']['sessionId']
get_route_distance(result)

107.686

In [ ]:
# Filter gas stations within 5 mi range of  Corridor
WIDTH = 5
gas_station = pd.read_csv('Data/wa_gas_stations.csv')
remoteDataList = getRemoteDataList(gas_station,'Latitude','Longitude','name')

result = post_corridor_search_request(sessionId, remoteDataList, 5)
print(result['resultsCount'])
filtered_gas_stations = pd.DataFrame(result['searchResults'])

259


In [ ]:
# covert exits to remoteData Format
exitRomoteDataList = getRemoteDataList(exits, 'geometry.y', 'geometry.x', 'attributes.ExitNumber')

In [ ]:
# for each gas station near I-5, compute its distance to closest exit
dist_to_exit = []
for gas_r in range(filtered_gas_stations.shape[0]):
  gas_data = filtered_gas_stations.iloc[gas_r]
  dist, exit_n, exit_p = get_closest_exit(gas_data['shapePoints'], exitRomoteDataList)
  # print(gas_data['resultNumber'], dist)
  dist_to_exit.append([gas_data['key'],gas_data['shapePoints'][0],gas_data['shapePoints'][1],exit_n,exit_p[0],exit_p[1],dist])
dist_to_exit = pd.DataFrame(dist_to_exit)
dist_to_exit.columns = ['gas_key','gas_lat','gas_long','exit_name','exit_lat','exit_long','distance_to_nearest_exit']

In [ ]:
dist_to_exit.to_csv('Data/gas_station_w_dist_to_exit_I5.csv')

In [ ]:
dist_to_exit = dist_to_exit.rename(columns={"distance": "distance_to_nearest_exit"})
dist_to_exit.to_csv('Data/gas_station_w_dist_to_exit_I5.csv')
dist_to_exit

,gas_key,gas_lat,gas_long,exit_name,exit_lat,exit_long,distance_to_nearest_exit
0,1101,46.009816,-122.845820,30,46.003649,-122.843998,0.4355
1,832,47.044527,-122.845099,108,47.044582,-122.843298,0.0850
2,1783,45.911880,-122.745401,21,45.917607,-122.749961,0.4528
3,1033,45.908302,-122.744860,21,45.902501,-122.740547,0.4518
4,488,46.710777,-122.974268,82,46.717613,-122.976120,0.4810
...,...,...,...,...,...,...,...
254,1743,45.677940,-122.576551,32,45.679332,-122.587012,0.5147
255,1592,45.677526,-122.575504,32,45.679332,-122.587012,0.5700
256,287,47.058038,-122.760441,111,47.064374,-122.759032,0.4433
257,1646,45.677604,-122.574833,32,45.679332,-122.587012,0.6006
